In [30]:
def occupancy(episodes, s, a, timestep_max, gamma):
    ''' 计算状态动作对（s,a）出现的频率,以此来估算策略的占用度量 '''
    rho = 0
    total_times = np.zeros(timestep_max)  # 记录每个时间步t各被经历过几次
    occur_times = np.zeros(timestep_max)  # 记录(s_t,a_t)=(s,a)的次数
    for episode in episodes:              ## 遍历每个采样的内容
        for i in range(len(episode)):     ## 遍历每个采样内部的状态、动作的内容
            (s_opt, a_opt, r, s_next) = episode[i]   ## 拿到每个采样内的 状态、动作、动作的奖励、下一个状态
            total_times[i] += 1                      ## 统计序列所在index的次数
            if s == s_opt and a == a_opt:            ## 状态、动作和给定的相同
                occur_times[i] += 1                  ## 也就是（状态，动作）对的次数+1
    for i in reversed(range(timestep_max)):       ## 逆序算占用度量
        if total_times[i]:                        ## 序列所在的index有值
            rho += gamma**i * occur_times[i] / total_times[i]        ## 按照公式运算
    return (1 - gamma) * rho


gamma = 0.5
timestep_max = 1000

episodes_1 = sample(MDP, Pi_1, timestep_max, 10000)  ## 采样，参数是MDP，Pi_1是策略，timestep_max是最长时间步，10000是采样的次数
episodes_2 = sample(MDP, Pi_2, timestep_max, 10000)  ## 和上面只有策略不同，Pi_2
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)    ## 算占用度量
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

# 0.112567796310472 0.23199480615618912

0.11712105221274086 0.2329241059970849
